In [1]:
# load_data.py
import pandas as pd

def load_dataset(file_path):
    df = pd.read_excel(file_path)
    return df

if __name__ == "__main__":
    df = load_dataset("AGB_Data.xlsx")
    print(df.head())

    NDVI  TNDVI     SR   SAVI  MSAVI2        AGB
0 -0.072  0.654  0.866 -0.108  -0.154   7.158685
1 -0.072  0.654  0.866 -0.108  -0.154  12.112405
2 -0.056  0.666  0.894 -0.084  -0.118  21.010835
3 -0.048  0.672  0.908 -0.072  -0.101  24.553987
4 -0.041  0.678  0.922 -0.061  -0.084  25.799105


In [2]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('AGB', axis=1)
y = df['AGB']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5/55, random_state=42)

print(f"train: {X_train.shape[0]} test: {X_test.shape[0]}")

train: 50 test: 5


In [3]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np # Import numpy for sqrt

# X_train, X_test, y_train, y_test = prepare_data("AGB_Data.xlsx")

model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
model.fit(X_train, y_train)
y_pred_svr = model.predict(X_test)

# Fix: Calculate RMSE by taking the square root of MSE instead of using squared=False
mse = mean_squared_error(y_test, y_pred_svr)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_svr)

print(f"SVM (RBF Kernel) -> R²: {r2:.4f}, RMSE: {rmse:.4f}")

SVM (RBF Kernel) -> R²: 0.8984, RMSE: 4.5985


In [4]:
import joblib
joblib.dump(model, "svr_agb_model.pkl")
loaded_model = joblib.load("svr_agb_model.pkl")
y_pred_loaded = loaded_model.predict(X_test)

In [5]:
image = "IMAGERY.TIF"

In [6]:
import rasterio

with rasterio.open(image) as src:
   print("CRS:", src.crs)            # ระบบพิกัดที่ไฟล์ใช้
   print("Bounds:", src.bounds)      # ขอบเขตพื้นที่ raster (ในหน่วย CRS)

CRS: EPSG:32647
Bounds: BoundingBox(left=475129.5290357446, bottom=1932627.81524834, right=586939.5290357446, top=2031777.81524834)


In [7]:
from pyproj import Transformer

lat, lon = 18.11302, 99.31819  # พิกัดผู้ใช้ (WGS84)
with rasterio.open(image) as src:
   raster_crs = src.crs  # EPSG:32647
   # แปลง WGS84 → CRS ของ raster
   transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)
   x, y = transformer.transform(lon, lat)
   # แปลง x,y → row, col
   row, col = src.index(x, y)
   print("CRS Coordinates:", x, y)
   print("Pixel row,col:", row, col)

CRS Coordinates: 533663.04412683 2002718.7692851895
Pixel row,col: 1937 3902


In [8]:
import rasterio

# ---- Load ML model ----
model = joblib.load("svr_agb_model.pkl")
# ---- Load raster ----
raster_path = image
src = rasterio.open(raster_path)
raster_crs = src.crs
# ---- CRS transformer: WGS84 -> raster CRS ----
transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)
# ---- Constants for indices ----
L = 0.5
# ---- พิกัดตัวอย่างจากผู้ใช้ ----
lat, lon = 18.11302, 99.31819
# ---- Transform lat/lon -> raster CRS ----
x, y = transformer.transform(lon, lat)
print(f"CRS Coordinates: {x} {y}")
# ---- Transform CRS -> pixel row/col ----
row, col = src.index(x, y)
print(f"Pixel row,col: {row} {col}")
# ---- ตรวจ bounds ----
if not (0 <= row < src.height and 0 <= col < src.width):
   raise ValueError("Coordinate is outside the raster bounds!")
# ---- Read pixel values ----
pixel = src.read()[:, row, col]  # [R,G,B,NIR]
R, G, B, NIR = pixel
print("Raw bands:", pixel)
# ---- Compute indices ----
ndvi   = (NIR - R) / (NIR + R + 1e-6)
tndvi  = np.sqrt((NIR - R) / (NIR + R + 1e-6) + L)
sr     = NIR / (R + 1e-6)
savi   = ((1 + L) * (NIR - R)) / (NIR + R + L + 1e-6)
msavi2 = (2 * NIR + 1 - np.sqrt((2 * NIR + 1)**2 - 8 * (NIR - R))) / 2
print(f"NDVI={ndvi:.3f}, TNDVI={tndvi:.3f}, SR={sr:.3f}, SAVI={savi:.3f}, MSAVI2={msavi2:.3f}")
# ---- Prepare feature vector สำหรับ ML ----
# ใช้เฉพาะ indices 5 ตัว ตามที่ train model
X = np.array([ndvi, tndvi, sr, savi, msavi2]).reshape(1, -1)
# ---- Predict AGB ----
agb_pred = model.predict(X)[0]
print(f"Predicted AGB={agb_pred:.3f}")
# ---- ตัวอย่าง JSON สำหรับส่ง frontend ----
result = {
   "coordinate": {"lat": lat, "lon": lon},
   "pixel": {"row": int(row), "col": int(col), "R": int(R), "G": int(G), "B": int(B), "NIR": int(NIR)},
   "indices": {"NDVI": float(ndvi), "TNDVI": float(tndvi), "SR": float(sr), "SAVI": float(savi), "MSAVI2": float(msavi2)},
   "AGB": float(agb_pred)
}
print(result)

CRS Coordinates: 533663.04412683 2002718.7692851895
Pixel row,col: 1937 3902
Raw bands: [55 60 67 74]
NDVI=0.147, TNDVI=0.805, SR=1.345, SAVI=0.220, MSAVI2=71.625
Predicted AGB=33.299
{'coordinate': {'lat': 18.11302, 'lon': 99.31819}, 'pixel': {'row': 1937, 'col': 3902, 'R': 55, 'G': 60, 'B': 67, 'NIR': 74}, 'indices': {'NDVI': 0.14728682056366807, 'TNDVI': 0.8045413728104155, 'SR': 1.3454545209917361, 'SAVI': 0.2200772183777821, 'MSAVI2': 71.625}, 'AGB': 33.29895154537698}


c:\Users\Acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
